In [2]:
from flask import Flask, request, jsonify
import openai
import json
import pg8000
from dotenv import load_dotenv
import os

app = Flask(__name__)

# Load environment variables
load_dotenv(dotenv_path="./.env")

# OpenAI API Key
openai.api_key = os.getenv("OPENAI_API_KEY")

# Database connection parameters
DATABASE_URL = os.getenv("POSTGRES_URL")
DATABASE_USER = os.getenv("POSTGRES_USER")
DATABASE_PASS = os.getenv("POSTGRES_PASSWORD")
DATABASE_DB = os.getenv("POSTGRES_DATABASE")
DATABASE_HOST = os.getenv("POSTGRES_HOST")


### Start with Connecting to DB, also functions for receiving from DB

In [7]:
def get_db_connection():
    """ Creates a Database connection """
    conn = pg8000.connect(
        user=DATABASE_USER,
        password=DATABASE_PASS,
        host=DATABASE_HOST,
        database=DATABASE_DB
    )
    return conn


def get_user_diet(uid, cursor):
    """
    Retrieves the User Diet Restrictions
    
    Inputs:\\
    uid - user Id\\
    cursor - database cursor object

    Returns:\\
    String of all assoicated dietary requirements
    """

    try:
        query = f"""
                SELECT dt."dietName" FROM userpreference as up
                JOIN diettype as dt ON up."dietId" = dt."dietId" 
                WHERE up.uid = {uid} 
                """
        cursor.execute(query)
        diets = cursor.fetchall()
        diet_list = [diet[0] for diet in diets]
        return ', '.join(diet_list)
    except Exception as e:
        print(f"Database error: {e}")
        return ""
    

def get_conversation_context(uid, cursor):
    """ 
    Gets context to give to chatbot 
    
    Inputs:\\
    uid - user Id\\
    cursor - database cursor object

    Returns:\\
    List of 10 most previous questions and answers formatted to give
    chatbot correct context
    """

    try:
        query = f"""
            SELECT "userQuestion", "chatResponse"
            FROM context
            WHERE uid = {uid}
            ORDER BY timestamp DESC
            LIMIT 10;
        """
        cursor.execute(query)
        context = cursor.fetchall()
        # Flatten the context into a list of messages
        messages = []
        for user_q, chat_resp in reversed(context):
            messages.append({"role": "user", "content": user_q})
            messages.append({"role": "assistant", "content": chat_resp})
        return messages
    except Exception as e:
        print(f"Database error: {e}")
        return []
    

def get_user_recipes(uid, cursor):
    """
    Retrieves the previous user recipes
    
    Inputs:\\
    uid - user Id\\
    cursor - database cursor object

    Returns:\\
    List of previous recipes names for context
    """

    try:
        query = f"""
            SELECT "recipeName"
            FROM recipe
            WHERE uid = {uid}; 
        """
        cursor.execute(query)
        recipes = cursor.fetchall()
        recipe_names = [recipe[0] for recipe in recipes]
        return recipe_names
    except Exception as e:
        print(f"Database error: {e}")
        return []


### Classify the message

In [16]:
def classify_message(user_question, user_diet, context, client):
    classification_rules = """
    You are to classify the user's request into one of the following categories:
    - "recipe": If the user is asking for a recipe.
    - "lifestyle": If the user is seeking lifestyle tips related to their diet.
    - "general": If the user has general questions about their dietary needs.
    - "not applicable": If the request is unrelated to diet or lifestyle.

    Respond with only one of the above categories.
    """

    try:
        completion = client.chat.completions.create(
            model="gpt-4o-mini", 
            messages=[
                {"role": "system", "content": classification_rules},
                {"role": "assistant", "content": f"Dietary preferences: {user_diet}, context: {context}"},
                {"role": "user", "content": user_question}
            ],
            max_tokens=5,
            n=1,
            stop=None,
            temperature=0
        )
        classification = completion.choices[0].message.content.strip().lower()
        return classification
    except Exception as e:
        print(f"Classification error: {e}")
        return None


In [17]:
conn = get_db_connection()
cursor = conn.cursor()
client = openai.OpenAI()

try:
    user_diet = get_user_diet(1, cursor)
    context = get_conversation_context(1, cursor)
    # recipes = get_user_recipes(1, cursor)

    user_message = "How can I avoid foods I can't eat in the supermarket?"

    classify = classify_message(user_message, user_diet, context, client)
    print(classify)

    # print(user_diet)
    # print(context)
    # print(recipes)

except Exception as e:
    print("Exception:" + e)

finally:
    conn.close()

lifestyle


In [89]:
conn = pg8000.connect(
    user=DATABASE_USER,
    password=DATABASE_PASS,
    host=DATABASE_HOST,
    database=DATABASE_DB
)
cursor = conn.cursor()
try:
    # Example query: Fetch User table
    cursor.execute('SELECT * FROM diettype')
    rows = cursor.fetchall()
    print(rows)

    # Example query: Fetch User table
    cursor.execute("""SELECT dt."dietName" FROM userpreference as up
                   JOIN diettype as dt ON up."dietId" = dt."dietId" 
                   WHERE up.uid = 1 """)
    rows = cursor.fetchall()
    print(rows)

    # # Fetch column names and their data types for each table
    # tables = ['users', 'UserPreference', 'DietType', 'Context', 'Tip', 'Recipe']

    # for table in tables:
    #     cursor.execute(f"""
    #         SELECT column_name, data_type 
    #         FROM information_schema.columns 
    #         WHERE table_name = '{table}';
    #     """)
    #     columns = cursor.fetchall()
    #     print(f"Columns and their types in '{table}' table:")
    #     for column_name, data_type in columns:
    #         print(f"  {column_name}: {data_type}")
    #     print()  # Blank line for readability
except Exception as e:
    print("Exception" + e)

    # Close the connection
finally:
    conn.close()

([1, 'GlutenFree'], [2, 'LowFodmap'], [3, 'Mediterranean'], [4, 'DASH'], [5, 'Anti-Inflammatory'], [6, 'LowCal-LowFat'])
(['GlutenFree'], ['Anti-Inflammatory'])
